In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/sentiment')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load train.csv and test.csv. Do the data preprocessing same with Training.

If you input new sentence, need TEXT to do data preprocessing.

In [2]:
import load_review
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import TabularDataset
from torchtext import data
from torchtext.vocab import GloVe
tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True,
                      fix_length=200)
LABEL = data.LabelField()
train_data, test_data = TabularDataset.splits(path='', train='train.csv', test='test.csv', format='csv',
                                                  fields=[('text', TEXT), ('label', LABEL)])
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)
word_embeddings = TEXT.vocab.vectors
print("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print("Label Length: " + str(len(LABEL.vocab)))

train_iter, test_iter = data.BucketIterator.splits((train_data, test_data), batch_size=32,
                                                        sort_key=lambda x: len(x.text), repeat=False,
                                                        shuffle=True)
vocab_size = len(TEXT.vocab)

Length of Text Vocabulary: 188707
Vector size of Text Vocabulary:  torch.Size([188707, 300])
Label Length: 3


Define bilstm_attm model.

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np


class bilstm_attn(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embed_dim, weights, bidirectional, dropout,
                 use_cuda, attention_size, sequence_length):
        super(bilstm_attn, self).__init__()

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.bidirectional = bidirectional
        # self.dropout = dropout
        self.use_cuda = use_cuda
        self.sequence_length = sequence_length
        self.lookup_table = nn.Embedding(self.vocab_size, self.embed_dim)
        # self.lookup_table.weight.data.uniform_(-1., 1.)
        self.lookup_table.weight = nn.Parameter(weights, requires_grad=False)

        self.layer_size = 1
        self.lstm = nn.LSTM(self.embed_dim,
                            self.hidden_size,
                            self.layer_size,
                            bidirectional=self.bidirectional)

        if self.bidirectional:
            self.layer_size = self.layer_size * 2
        else:
            self.layer_size = self.layer_size

        self.attention_size = attention_size
        if self.use_cuda:
            self.w_omega = Variable(torch.zeros(self.hidden_size * self.layer_size, self.attention_size).cuda())
            self.u_omega = Variable(torch.zeros(self.attention_size).cuda())
        else:
            self.w_omega = Variable(torch.zeros(self.hidden_size * self.layer_size, self.attention_size))
            self.u_omega = Variable(torch.zeros(self.attention_size))

        self.label = nn.Linear(hidden_size * self.layer_size *3, output_size)
        self.dropout = nn.Dropout(dropout)
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.softmax = nn.Softmax(dim=1)
        self.init_weights()
        # init_lstm( self.lstm )
   

    # self.attn_fc_layer = nn.Linear()

    def init_weights(self):
      initrange = 0.5
      self.label.weight.data.uniform_(-initrange, initrange)
      self.label.bias.data.zero_()

    def attention_net(self, lstm_output):
        # print(lstm_output.size()) = (squence_length, batch_size, hidden_size*layer_size)

        output_reshape = torch.Tensor.reshape(lstm_output, [-1, self.hidden_size * self.layer_size])
        # print(output_reshape.size()) = (squence_length * batch_size, hidden_size*layer_size)

        attn_tanh = torch.tanh(torch.mm(output_reshape, self.w_omega))
        # print(attn_tanh.size()) = (squence_length * batch_size, attention_size)

        attn_hidden_layer = torch.mm(attn_tanh, torch.Tensor.reshape(self.u_omega, [-1, 1]))
        # print(attn_hidden_layer.size()) = (squence_length * batch_size, 1)

        exps = torch.Tensor.reshape(torch.exp(attn_hidden_layer), [-1, self.sequence_length])
        # print(exps.size()) = (batch_size, squence_length)

        alphas = exps / torch.Tensor.reshape(torch.sum(exps, 1), [-1, 1])
        # print(alphas.size()) = (batch_size, squence_length)

        alphas_reshape = torch.Tensor.reshape(alphas, [-1, self.sequence_length, 1])
        # print(alphas_reshape.size()) = (batch_size, squence_length, 1)

        state = lstm_output.permute(1, 0, 2)
        # print(state.size()) = (batch_size, squence_length, hidden_size*layer_size)

        attn_output = torch.sum(state * alphas_reshape, 1)
        # print(attn_output.size()) = (batch_size, hidden_size*layer_size)

        return attn_output

    def forward(self, input_sentences, batch_size=None):
        input = self.lookup_table(input_sentences)
        input = self.dropout(input)
        input = input.permute(1, 0, 2)
        if batch_size is None:
          current_batch_size = self.batch_size
        else:
          current_batch_size = batch_size

        if self.use_cuda:
            h_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size))
            c_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size))

        lstm_output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        lstm_output = self.dropout(lstm_output)
        max_p = self.max_pool(lstm_output.permute(1,2,0)).squeeze(2)
        avg_p = self.avg_pool(lstm_output.permute(1,2,0)).squeeze(2)
        attn_output = self.attention_net(lstm_output)
        x= torch.cat((attn_output,max_p,avg_p),dim=1)
        logits = self.label(x)
        # output = self.softmax(logits)
        return logits,attn_output


Model initialize.
You can change the model_file with own model filename.

I named them like 'best_model_hidden_128_eval_weighted.pt' or 'best_model_hidden_128_eval.pt'


In [8]:
lr=1e-3
batch_size = 32
output_size = 3
hidden_size = 128
embedding_length = 300
num_layers=1
seed=12
dropout=0.5
torch.manual_seed(seed)
use_cuda = torch.cuda.is_available()
sequence_length=200 
attention_size=200
model = bilstm_attn(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings, True, dropout,
                    use_cuda, sequence_length, attention_size)
model_file='best_model_hidden_128_eval_weighted.pt'
model.load_state_dict(torch.load(model_file))
model.cuda()
model.eval()

bilstm_attn(
  (lookup_table): Embedding(188707, 300)
  (lstm): LSTM(300, 128, bidirectional=True)
  (label): Linear(in_features=768, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (max_pool): AdaptiveMaxPool1d(output_size=1)
  (avg_pool): AdaptiveAvgPool1d(output_size=1)
  (softmax): Softmax(dim=1)
)

function pad() to add <pad> aiming for keeping the input sentence length to 200.
You can change the test_sen1 to your own sentence.
It also prints the probabilities that a sample belonging to each category.

For example: [0.7841, 0.0466, 0.1693] => [Positive , Negative, Neutral]


**Notice**

Labels in train_data and test_data are different with the original dataset.

Torchtext helps to change the label accoring to the different class proporation.(positive > negative > neutral)

Now 0 means positive. 1 means negative. 2 means neutral.

In [10]:
def pad(TEXT, x):
  if not TEXT.sequential:
    return minibatch
  if TEXT.fix_length is None:
    max_len = max(len(x) for x in minibatch)
  else:
    max_len = TEXT.fix_length + (
    TEXT.init_token, TEXT.eos_token).count(None) - 2
    padded, lengths = [], []
  if TEXT.pad_first:
    padded.append([TEXT.pad_token] * max(0, max_len - len(x)) + ([] if TEXT.init_token is None else [TEXT.init_token])+ list(x[-max_len:] if TEXT.truncate_first else x[:max_len])+ ([] if TEXT.eos_token is None else [TEXT.eos_token]))
  else:
    padded.append(([] if TEXT.init_token is None else [TEXT.init_token])+ list(x[-max_len:] if TEXT.truncate_first else x[:max_len])+ ([] if TEXT.eos_token is None else [TEXT.eos_token])+ [TEXT.pad_token] * max(0, max_len - len(x)))
  return padded

test_sen1 = "Great price and cute color  Better price than in stores  Just a rubber case wont protect phone if dropped hard"
print(test_sen1)
test_sen1 = TEXT.preprocess(test_sen1)
len1=len(test_sen1)
test_sen1 = pad(TEXT,test_sen1)[0]
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]
test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = Variable(test_sen,requires_grad=False)
test_tensor = test_tensor.cuda()
output,att = model(test_tensor, 1)
output = F.softmax(output, 1)
print(output.data)
if torch.argmax(output[0]) == 0:
    print ("Sentiment: Positive")
elif  torch.argmax(output[0]) == 2:
    print ("Sentiment: Neutral")
else:
    print ("Sentiment: Negative")
# att=torch.split(att, len1, dim=1)[0]
# att = F.softmax(att, 1)
# print(att.data.cpu().numpy().tolist())

Great price and cute color  Better price than in stores  Just a rubber case wont protect phone if dropped hard
tensor([[0.7841, 0.0466, 0.1693]], device='cuda:0')
Sentiment: Positive


Get label list and predicted list. 

In [0]:

def eval_model(model,val_iter):
  total_epoch_loss = 0
  total_epoch_acc = 0
  cnt=0
  model.eval()
  label_list=[]
  pred_list=[]
  with torch.no_grad():
      for idx, batch in enumerate(val_iter):
          text = batch.text[0]
          # for sen in text.numpy().tolist():
            # s=[]
            # for w in sen:
            #   s.append(id2str[w])
            # print(s)
          if (text.size()[0] is not 32):
              continue
          target = batch.label
          target = torch.autograd.Variable(target).long()
          if torch.cuda.is_available():
              text = text.cuda()
              target = target.cuda()
          prediction,_ = model(text)
          num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
          label_list += target.data.cpu().numpy().tolist()
          pred_list+=torch.max(prediction, 1)[1].view(target.size()).data.cpu().numpy().tolist()
          acc = 100.0 * num_corrects/len(batch)
          total_epoch_acc += acc.item()
  return  total_epoch_acc/len(val_iter),label_list,pred_list

_,label_list,predict_list=eval_model(model,test_iter)


Print the f1-score, precision, recall, accuracy.

In report, the performance of the balanced BiSLTM model is as belows:

f1-score => 0.670272189314557

precision => 0.6453988845119311

recall => 0.71894965957209

accuracy => 0.8000104807154835

In [91]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

print(predict_list)
print(label_list)
def get_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, labels=[0,1,2], average='macro')

def get_precision_score(y_true, y_pred):
    return precision_score(y_true, y_pred, labels=[0,1,2], average='macro')

def get_recall_score(y_true, y_pred):
    return recall_score(y_true, y_pred, labels=[0,1,2], average='macro')

def get_acc_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

print(get_f1_score(label_list,predict_list))
print(get_precision_score(label_list,predict_list))
print(get_recall_score(label_list,predict_list))
print(get_acc_score(label_list,predict_list))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 